In [1]:
import functions as fn
import pandas as pd
from importlib import reload
from tqdm import tqdm
from termcolor import colored
import matplotlib.pyplot as plt
from ooipy.tools import ooiplotlib as ooiplt
import datetime

In [2]:
df = pd.read_csv('E:/Acer backup 3/internships and jobs/UW/Reader grader and TA/Shima Abadi/Data/oregon_hydrate.csv',sep = ',')


In [3]:
df.columns

Index(['Unnamed: 0', 'MMSI', 'BaseDateTime', 'LAT', 'LON', 'SOG', 'COG',
       'Heading', 'VesselName', 'IMO', 'CallSign', 'Status', 'LENGTH', 'Width',
       'Draft', 'Cargo', 'TransceiverClass', 'TIMESTAMP UTC',
       'oregon_slope_Lat', 'oregon_slope_Lon', 'oregon_slope_Loc', 'ship_Loc',
       'Southern_hydrate_Lat', 'Southern_hydrate_lon', 'Southern_hydrate_Loc',
       'distance(in km) oregon', 'distance(in km) hydrate', 'Type Code',
       'VESSEL TYPE', 'Group', 'group3'],
      dtype='object')

In [4]:
df['TIMESTAMP UTC']=df['TIMESTAMP UTC'].astype('datetime64[ns]')


In [5]:
df.head()

,Unnamed: 0,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,...,ship_Loc,Southern_hydrate_Lat,Southern_hydrate_lon,Southern_hydrate_Loc,distance(in km) oregon,distance(in km) hydrate,Type Code,VESSEL TYPE,Group,group3
0,0,636092819,2021-01-01T00:17:40,44.56820,-125.62248,11.0,180.6,181.0,NaN,NaN,...,"(44.5682, -125.62248)",44.5691,-125.1481,"(44.5691, -125.1481)",19.40,37.68,NaN,NaN,NaN,NaN
1,1,636092819,2021-01-01T00:19:00,44.56414,-125.62273,11.1,187.9,180.0,NaN,NaN,...,"(44.56414, -125.62273)",44.5691,-125.1481,"(44.5691, -125.1481)",19.29,37.71,NaN,NaN,NaN,NaN
2,2,636092819,2021-01-01T00:20:08,44.56068,-125.62271,11.4,162.9,179.0,NaN,NaN,...,"(44.56068, -125.62271)",44.5691,-125.1481,"(44.5691, -125.1481)",19.19,37.72,NaN,NaN,NaN,NaN
3,3,636092819,2021-01-01T00:22:19,44.55397,-125.62281,10.9,186.6,182.0,NaN,NaN,...,"(44.55397, -125.62281)",44.5691,-125.1481,"(44.5691, -125.1481)",19.01,37.75,NaN,NaN,NaN,NaN
4,4,636092819,2021-01-01T00:24:00,44.54881,-125.62310,10.8,177.9,180.0,NaN,NaN,...,"(44.54881, -125.6231)",44.5691,-125.1481,"(44.5691, -125.1481)",18.91,37.81,NaN,NaN,NaN,NaN


In [6]:
df['TIMESTAMP UTC'].min()

Timestamp('2017-01-01 09:48:46')

In [7]:
df['TIMESTAMP UTC'].max()

Timestamp('2022-06-30 23:59:29')

In [8]:
df['VESSEL TYPE'].unique()

array([nan, 'Cargo, all ships of this type', 'Pleasure Craft', 'Fishing',
       'Sailing', 'Passenger, all ships of this type',
       'Tanker, all ships of this type',
       'Other Type, all ships of this type',
       'Cargo, Hazardous category A', 'Towing',
       'Cargo, Hazardous category C',
       'Other Type, no additional information',
       'Cargo, No additional information', 'Spare - Local Vessel',
       'Tanker, No additional information', 'Cargo, Hazardous category B',
       'Cargo, Hazardous category D', 'Tanker, Hazardous category B',
       'Tanker, Hazardous category A', 'Tug', 'Not available (default)',
       'Wing in ground (WIG), all ships of this type',
       'Other Type, Hazardous category A', 'Tanker, Hazardous category D',
       'Military ops',
       'Towing: length exceeds 200m or breadth exceeds 25m',
       'Tanker, Hazardous category C',
       'Passenger, No additional information'], dtype=object)

In [9]:
min_duration=10
path_to_write='E:/Acer backup 3/internships and jobs/UW/Reader grader and TA/Shima Abadi/Data/Benchmark_data/'

In [10]:
reload(fn)

<module 'functions' from 'C:\\Users\\khirod\\Documents\\GitHub\\Ocean-data-analysis\\functions.py'>

In [11]:
def get_benchmark_data(df,hydrophone_idx,inner_rad,outer_rad,min_duration,fmin=10,fmax=90,break_dur=True,get_acoustic=True,get_spectrogram=True):
    
    if hydrophone_idx==1:
        hydro='Oregon_slope'
    elif hydrophone_idx==2:
        hydro='Southern_hydrate'
    suffix=str(inner_rad)+'_'+ str(outer_rad)+'.csv'
    ais=fn.choose_df_slope_hydrate(df,hydrophone_idx)
    isolated_ships=fn.get_isolated_ships(ais,inner_rad,outer_rad,min_duration)
    isolated_ships.to_csv(path_to_write+hydro+'/isolated_ships_' + suffix)
    isolated_ais=fn.isolated_ais(ais,isolated_ships,inner_rad)
    isolated_ais.to_csv(path_to_write+hydro+'/isolated_ais_'+ suffix)
    if break_dur:
        isolated_ais_10m=fn.break_duration(isolated_ships)
        isolated_ais_10m.to_csv(path_to_write+hydro+'/isolated_ais_10m_'+ suffix)
    
    if get_acoustic==True:
        print('Saving wav files')
        for i in tqdm(range(len(isolated_ships))):
            min_time=isolated_ships.start_time[i]
            max_time=isolated_ships.end_time[i]
            hydrophone_data=fn.get_acoustic(hydrophone_idx,min_time,max_time,fmin=fmin,fmax=fmax)
            if hydrophone_data==None:
                # print('data trace is none. Continuing to next')
                pass
            else:
                hydrophone_data.wav_write(filename=path_to_write+hydro+'/Audio'+'/isolated_ships_'+str(inner_rad)+'_'+ str(outer_rad)+'_'+str(i)+'.wav')
    if get_spectrogram==True:
        print('Saving spectrograms')
        for i in tqdm(range(len(isolated_ships))):
            min_time=isolated_ships.start_time[i]
            max_time=isolated_ships.end_time[i]
            spectrogram=fn.get_spectrogram_data(hydrophone_idx,min_time,max_time,fmin=fmin,fmax=fmax)
            if (spectrogram==None):
                # print('data trace is none. Continuing to next')
                pass
            else:
                if (spectrogram.values.shape[0]<2) | (spectrogram.values.shape[1]<2):
                    pass
                else:
                    #spectrogram.visualize(save_spec=True,plot_spec=False,filename=path_to_write+hydro+'/Spectrogram'+'/isolated_ships_'+str(inner_rad)+'_'+ str(outer_rad)+'_'+str(i)+'.png')
                    ooiplt.plot_spectrogram(spectrogram,plot=False,save=True,fmin=fmin,fmax=fmax,filename=path_to_write+hydro+'/Spectrogram'+'/isolated_ships_'+str(inner_rad)+'_'+ str(outer_rad)+'_'+str(i)+'.png')
    return isolated_ships,isolated_ais

## Creating benchmark data for Oregon slope

### inner rad=10, outer_rad=30

In [28]:

d1,d2=get_benchmark_data(df,1,10,30,min_duration,get_acoustic=False,get_spectrogram=False)

 Max distance: 57.89 and Min distance: 0.0


100%|██████████████████████████████████████████████████████████████████████████████| 1105/1105 [00:21<00:00, 52.10it/s]


### Depth=2.9
### Oregon slope= 'HYSB1'

### inner rad=2.9, outer_rad=2.9, outer_rad=5.8

In [43]:
inner_rad= 6
outer_rad1=24
outer_rad2=5.8

In [44]:
d1,d2=get_benchmark_data(df,1,inner_rad,outer_rad1,min_duration,break_dur=False,get_acoustic=False,get_spectrogram=False)

 Max distance: 57.89 and Min distance: 0.0


100%|████████████████████████████████████████████████████████████████████████████████| 428/428 [00:06<00:00, 70.95it/s]


In [31]:
d1

,MMSI,start_time,end_time,count,VESSEL TYPE,len_of_recording
0,235010450,2017-05-23 18:40:59,2017-05-23 18:51:59,11,NaN,11.000000
1,235010450,2017-07-05 06:37:19,2017-07-05 06:47:19,10,NaN,10.000000
2,246028000,2017-09-27 01:59:36,2017-09-27 02:11:12,10,NaN,11.600000
3,255805864,2017-01-27 13:58:44,2017-01-27 14:10:01,11,NaN,11.283333
4,255805864,2017-04-01 23:34:25,2017-04-01 23:49:03,11,NaN,14.633333
...,...,...,...,...,...,...
65,477280800,2017-10-03 10:15:33,2017-10-03 10:29:32,9,"Cargo, all ships of this type",13.983333
66,477280800,2017-10-27 08:04:16,2017-10-27 08:18:06,8,"Cargo, all ships of this type",13.833333
67,538008492,2022-03-08 06:04:29,2022-03-08 06:16:36,12,"Cargo, all ships of this type",12.116667
68,538071203,2021-11-13 13:08:47,2021-11-13 13:21:19,9,Sailing,12.533333


In [45]:
print(d1['VESSEL TYPE'].value_counts(),'\n',d1['len_of_recording'].sum())

Cargo, all ships of this type         152
Other Type, all ships of this type     60
Towing                                 19
Tanker, all ships of this type         17
Fishing                                 8
Pleasure Craft                          5
Sailing                                 4
Not available (default)                 3
Passenger, all ships of this type       1
Tanker, No additional information       1
Spare - Local Vessel                    1
Cargo, Hazardous category A             1
Cargo, No additional information        1
Name: VESSEL TYPE, dtype: int64 
 24582.9


In [17]:
d1,d2=get_benchmark_data(df,1,inner_rad,outer_rad2,min_duration,get_acoustic=False,get_spectrogram=False)

 Max distance: 57.89 and Min distance: 0.0


100%|█████████████████████████████████████████████████████████████████████████████████| 84/84 [00:00<00:00, 128.47it/s]


{'MMSI': 235010450, 'start_time': Timestamp('2017-07-05 06:37:19'), 'end_time': Timestamp('2017-07-05 06:47:19'), 'VESSEL TYPE': nan, 'len_of_recording': 10.0}


In [18]:
print(d1['VESSEL TYPE'].value_counts(),'\n',d1['len_of_recording'].sum())

Other Type, all ships of this type    22
Towing                                 5
Sailing                                3
Cargo, all ships of this type          3
Fishing                                2
Pleasure Craft                         1
Name: VESSEL TYPE, dtype: int64 
 26752.316666666666


In [20]:
d1,d2=get_benchmark_data(df,1,inner_rad,outer_rad2,min_duration,get_acoustic=False,get_spectrogram=False)

 Max distance: 57.89 and Min distance: 0.0


100%|██████████████████████████████████████████████████████████████████████████████████| 84/84 [00:01<00:00, 56.55it/s]


{'MMSI': 235010450, 'start_time': Timestamp('2017-07-05 06:37:19'), 'end_time': Timestamp('2017-07-05 06:47:19'), 'VESSEL TYPE': nan, 'len_of_recording': 10.0}


In [ ]:
d1.head()

## Creating benchmark data for Southern hydrate

### Depth=772
### Southern hydrate= 'HYS14'

### inner rad=0.772, outer_rad=0.772, outer_rad=1.544

In [38]:
inner_rad= 1.5
outer_rad1=6
outer_rad2= 1.544

In [39]:
d1,d2=get_benchmark_data(df,2,inner_rad,outer_rad1,min_duration,break_dur=False,get_acoustic=False,get_spectrogram=False)

 Max distance: 76.26 and Min distance: 0.0


100%|██████████████████████████████████████████████████████████████████████████████████| 75/75 [00:01<00:00, 66.38it/s]


In [40]:
print(d1['VESSEL TYPE'].value_counts(),'\n',d1['len_of_recording'].sum())

Other Type, all ships of this type    20
Fishing                                4
Towing                                 1
Name: VESSEL TYPE, dtype: int64 
 26216.75


In [24]:
d1,d2=get_benchmark_data(df,2,inner_rad,outer_rad2,min_duration,get_acoustic=False,get_spectrogram=False)

 Max distance: 76.26 and Min distance: 0.0


100%|██████████████████████████████████████████████████████████████████████████████████| 43/43 [00:01<00:00, 33.97it/s]


{'MMSI': 319005600, 'start_time': Timestamp('2018-08-05 02:39:55'), 'end_time': Timestamp('2018-08-05 02:49:55'), 'VESSEL TYPE': nan, 'len_of_recording': 10.0}


In [25]:
print(d1['VESSEL TYPE'].value_counts(),'\n',d1['len_of_recording'].sum())

Other Type, all ships of this type    13
Name: VESSEL TYPE, dtype: int64 
 26585.95


In [26]:
if (spectrogram.values.shape[0]<2) | (spectrogram.values.shape[1]<2):
    pass
else:
    ooiplt.plot_spectrogram(spectrogram,plot=False,save=True,fmin=fmin,fmax=fmax,filename='test.png')

NameError: name 'spectrogram' is not defined

In [ ]:
# from functions import choose_df
# ais=choose_df_slope_hydrate(df,1)
# min_duration=20
# lat=float(ais['axial_Loc'].iloc[0][1:8])
# lon=float(ais['axial_Loc'].iloc[0][9:18])
# reload(fn)
# fn.get_isolated_map_plot('scatter',ais,10,30,lat,lon,min_duration)

In [ ]:
# reload(fn)
# fn.get_single_isolated_map_plot(1,'scatter',ais,10,30,lat,lon,min_duration,10,90)

## Comparison of various options

In [36]:
inner_rad=[1.5,3,6]
outer_rad=[3,6,12,24]
min_duration=10


In [37]:
reload(fn)

print(colored('Comparative view with minimum duration: '+ str(min_duration),'green',attrs=['bold']))
table=pd.DataFrame(columns=['hydrophone_idx','inner_rad','outer_rad','no_unique_ships','no_unique_vessels','no_records'])
for i in tqdm(range(1,3,1)):
    for i_r in tqdm(inner_rad):
        for o_r in outer_rad:
            if i_r<=o_r:
                temp_dict=dict()
                ais=fn.choose_df_slope_hydrate(df,i,verbose=False)

                temp_df1=fn.get_isolated_ships(ais,i_r,o_r,min_duration)
#                 temp_df2=fn.break_duration(temp_df1)
                temp_df2=temp_df1.copy()
                temp_dict['hydrophone_idx']=i
                temp_dict['inner_rad']=i_r
                temp_dict['outer_rad']=o_r
                temp_dict['no_unique_ships']=temp_df2['MMSI'].nunique()
                temp_dict['no_unique_vessels']=temp_df2['VESSEL TYPE'].nunique()
                temp_dict['no_records']=len(temp_df2)
                temp_dict['len_of_recordings']=temp_df2.len_of_recording.sum()
                table=table.append(temp_dict,ignore_index=True)
         
table


Comparative view with minimum duration: 10


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:09<00:00,  4.81s/it]


,hydrophone_idx,inner_rad,outer_rad,no_unique_ships,no_unique_vessels,no_records,len_of_recordings
0,1.0,1.5,3.0,8.0,2.0,59.0,11848.166667
1,1.0,1.5,6.0,8.0,2.0,61.0,11791.500000
2,1.0,1.5,12.0,8.0,2.0,68.0,10664.533333
3,1.0,1.5,24.0,7.0,2.0,50.0,7670.000000
4,1.0,3.0,3.0,37.0,6.0,75.0,28733.916667
5,1.0,3.0,6.0,37.0,6.0,89.0,26897.366667
6,1.0,3.0,12.0,36.0,6.0,106.0,24899.816667
7,1.0,3.0,24.0,31.0,6.0,105.0,16487.883333
8,1.0,6.0,6.0,221.0,13.0,494.0,44205.150000
9,1.0,6.0,12.0,211.0,13.0,498.0,35560.433333


In [22]:
reload(fn)

print(colored('Overlapping of all hydrophones with minimum number of timestamps: '+ str(min_duration),'green',attrs=['bold']))
table2=pd.DataFrame(columns=['inner_rad','outer_rad','no_unique_ships','no_unique_vessels','no_records'])

for i_r in inner_rad:
    for o_r in outer_rad:
        combined_df=pd.DataFrame()
        for i in range(1,4,1):
            temp_dict=dict()
            ais=fn.choose_df(df,i,verbose=False)
            d1=fn.get_isolated_ships(ais,i_r,o_r,min_duration)
            combined_df=pd.concat([combined_df,d1])

        temp_dict['inner_rad']=i_r
        temp_dict['outer_rad']=o_r
        temp_dict['no_unique_ships']=combined_df['MMSI'].nunique()
        temp_dict['no_unique_vessels']=combined_df['VESSEL TYPE'].nunique()
        temp_dict['no_records']=len(combined_df)
        table2=table2.append(temp_dict,ignore_index=True)
table2
        

Overlapping of all hydrophones with minimum number of timestamps: 20


,inner_rad,outer_rad,no_unique_ships,no_unique_vessels,no_records
0,5,20,88,19,524
1,5,30,82,19,558
2,5,40,73,19,613
3,10,20,478,37,1374
4,10,30,447,36,1362
5,10,40,396,36,1341
6,15,20,839,42,2683
7,15,30,797,42,2607
8,15,40,718,41,2440


In [ ]:
reload(fn)
df['TIMESTAMP UTC']=df['TIMESTAMP UTC'].astype('datetime64[ns]')
ais = fn.choose_df(df,1)
pings= fn.ais_ping_distribution(ais,show=False)